In [1]:
from isaacgym import gymtorch, gymapi
from isaacgymenvs import train
from rl_games.torch_runner import Runner
from hydra import compose, initialize
from rl_games.common import env_configurations, vecenv
from isaacgymenvs.utils.rlgames_utils import RLGPUEnv, RLGPUTaskAlgoObserver, get_rlgames_env_creator
from rl_games.algos_torch import model_builder
from isaacgymenvs.learning import common_player
from isaacgymenvs.utils.reformat import omegaconf_to_dict

import isaacgymenvs
import yaml
import torch
import numpy as np
import time
import os

os.environ['DISPLAY'] = ":5"

Importing module 'gym_37' (/scr-ssd/ksrini/Downloads/isaacgym/python/isaacgym/_bindings/linux-x86_64/gym_37.so)
Setting GYM_USD_PLUG_INFO_PATH to /scr-ssd/ksrini/Downloads/isaacgym/python/isaacgym/_bindings/linux-x86_64/usd/plugInfo.json
PyTorch version 1.8.1
Device count 1
/scr-ssd/ksrini/Downloads/isaacgym/python/isaacgym/_bindings/src/gymtorch
Using /afs/cs.stanford.edu/u/ksrini/.cache/torch_extensions as PyTorch extensions root...
Emitting ninja build file /afs/cs.stanford.edu/u/ksrini/.cache/torch_extensions/gymtorch/build.ninja...
Building extension module gymtorch...
Allowing ninja to set a default number of workers... (overridable by setting the environment variable MAX_JOBS=N)
ninja: no work to do.
Loading extension module gymtorch...


2022-07-26 08:32:10,865 - INFO - logger - logger initialized


Error: FBX library failed to load - importing FBX data will not succeed. Message: No module named 'fbx'
FBX tools must be installed from https://help.autodesk.com/view/FBX/2020/ENU/?guid=FBX_Developer_Help_scripting_with_python_fbx_installing_python_fbx_html


## Initialize config and defining builder helper functions

In [2]:
with initialize(config_path="cfg", job_name="test_env"):
    cfg = compose(config_name="config", overrides=["task=AllegroHandGrasp", 
                                                   "task.env.observationType=full_state",
                                                   # "task.env.objectType=spray_bottle",
                                                   "sim_device=cpu",
                                                   "headless=false",
                                                   "test=true",
                                                   "task.env.useRelativeControl=false",
                                                   "num_envs=1"])


def create_env_thunk(**kwargs):
        envs = isaacgymenvs.make(cfg.seed, cfg.task_name, cfg.task.env.numEnvs, 
            cfg.sim_device, cfg.rl_device, cfg.graphics_device_id, cfg.headless,
            cfg.multi_gpu, cfg.capture_video, cfg.force_render, cfg,
            **kwargs,
        )
        if cfg.capture_video:
            envs.is_vector_env = True
            envs = gym.wrappers.RecordVideo(
                envs,
                f"videos/{run_name}",
                step_trigger=lambda step: step % cfg.capture_video_freq == 0,
                video_length=cfg.capture_video_len,
            ) 
        return envs

# register new AMP network builder and agent
def build_runner(algo_observer):
    runner = Runner(algo_observer)
    runner.player_factory.register_builder(
        'a2c_continuous', lambda **kwargs : common_player.CommonPlayer(**kwargs)
    )
    return runner
        
# register the rl-games adapter to use inside the runner
vecenv.register('RLGPU',
                lambda config_name, num_actors, **kwargs: RLGPUEnv(config_name, num_actors, **kwargs))
env_configurations.register('rlgpu', {
    'vecenv_type': 'RLGPU',
    'env_creator': create_env_thunk,
})

/scr1/.pyenv/versions/miniconda3-latest/envs/rlgpu/lib/python3.7/site-packages/hydra/_internal/defaults_list.py:251: UserWarning: In 'config': Defaults list is missing `_self_`. See https://hydra.cc/docs/upgrades/1.0_to_1.1/default_composition_order for more information
  warnings.warn(msg, UserWarning)
/scr1/.pyenv/versions/miniconda3-latest/envs/rlgpu/lib/python3.7/site-packages/hydra/_internal/defaults_list.py:413: UserWarning: In config: Invalid overriding of hydra/job_logging:
Default list overrides requires 'override' keyword.
See https://hydra.cc/docs/next/upgrades/1.0_to_1.1/defaults_list_override for more information.

  deprecation_warning(msg)


## Create runner and player (agent)

In [3]:
runner = build_runner(RLGPUTaskAlgoObserver())
rlg_config_dict = omegaconf_to_dict(cfg.train)
runner.load(rlg_config_dict)
runner.reset()

self.seed = 42


In [4]:
agent = runner.create_player()
# agent.restore(cfg.checkpoint)
agent.games_num = agent.env.num_environments*10

Obs type: full_state
GPU Pipeline can only be used with GPU simulation. Forcing CPU Pipeline.
Num hand dofs: 

/scr1/.pyenv/versions/miniconda3-latest/envs/rlgpu/lib/python3.7/site-packages/gym/spaces/box.py:74: UserWarning: WARN: Box bound precision lowered by casting to float32
  "Box bound precision lowered by casting to {}".format(self.dtype)


Not connected to PVD
Physics Engine: PhysX
Physics Device: cpu
GPU Pipeline: disabled
 22
Num dofs:  24
{'observation_space': Box([-inf -inf -inf -inf -inf -inf -inf -inf -inf -inf -inf -inf -inf -inf
 -inf -inf -inf -inf -inf -inf -inf -inf -inf -inf -inf -inf -inf -inf
 -inf -inf -inf -inf -inf -inf -inf -inf -inf -inf -inf -inf -inf -inf
 -inf -inf -inf -inf -inf -inf -inf -inf -inf -inf -inf -inf -inf -inf
 -inf -inf -inf -inf -inf -inf -inf -inf -inf -inf -inf -inf -inf -inf
 -inf -inf -inf -inf -inf -inf -inf -inf -inf -inf -inf -inf -inf -inf
 -inf -inf -inf -inf -inf -inf -inf -inf -inf -inf -inf -inf -inf -inf
 -inf -inf -inf -inf -inf -inf], [inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf
 inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf
 inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf
 inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf
 inf inf inf inf inf inf inf inf inf inf inf inf

```python
runner.run_play({"train": False,
                 "play": True,
                 "checkpoint": cfg.checkpoint,
                 "sigma": None})
```

In [5]:
gym = agent.env.gym
env = agent.env

In [6]:
hand_handle = gym.get_actor_handle(env.envs[0], 0)
obj_handle = gym.get_actor_handle(env.envs[0], 1)
goal_obj_handle = gym.get_actor_handle(env.envs[0], 2)
hand_rb_dict = gym.get_actor_rigid_body_dict(env.envs[0], hand_handle)
obj_rb_dict = gym.get_actor_rigid_body_dict(env.envs[0], obj_handle)
goal_obj_rb_dict = gym.get_actor_rigid_body_dict(env.envs[0], goal_obj_handle)

In [7]:
import time

from isaacgym import torch_utils

def get_action(t):
    actions =  torch_utils.unscale(env.shadow_hand_dof_default_pos,
                                   env.shadow_hand_dof_lower_limits, 
                                   env.shadow_hand_dof_upper_limits).numpy()
    actions = actions[None, :]
    t += 6
    return torch.tensor(actions, device=env.device).float()

In [8]:
action = env.rb_forces.clone()
action[:, env.shadow_hand_rb_handles, 2] = 1
# action[:, env.object_rb_handles[1], 2] = -1.

# action = env.rb_forces.clone()

env.reset_idx([0], [0])  # resets
print(env.shadow_hand_dof_pos[0])
# action[:, hand_rb_dict['allegro_mount'], 2] = 1.4
gym.apply_rigid_body_force_tensors(env.sim, gymtorch.unwrap_tensor(action),
                                   gymtorch.unwrap_tensor(action),
                                   gymapi.ENV_SPACE)

tensor([-0.0600, -0.0600, -0.0600, -0.6280, -0.6280, -0.6280, -0.1117, -0.0558,
        -0.0558, -0.0558, -0.1117, -0.0558, -0.0558, -0.0558, -0.1117, -0.0558,
        -0.0558, -0.0558,  0.0558, -0.0663, -0.0558, -0.0558])


True

In [9]:
hand_joint_dict = gym.get_actor_joint_dict(env.envs[0], hand_handle)
gym.get_asset_joint_type(env.shadow_hand_asset, hand_joint_dict['allegro_mount_joint_rev_z'])

JointType.JOINT_REVOLUTE

In [10]:
env.reset_idx([0], [0])

In [20]:
t = 0
env.reset_idx([0], [0])  # resets

while t < 500:
    t += 1
    actions = env.shadow_hand_dof_pos
    obs, r, done, info = env.step(actions, rb_forces)
    # gym.apply_rigid_body_force_tensors(env.sim, gymtorch.unwrap_tensor(action),
    #                                gymtorch.unwrap_tensor(action),
    #                                gymapi.ENV_SPACE)
    env_ids = done.nonzero(as_tuple=False).squeeze(-1)
    goal_env_ids = env.reset_goal_buf.nonzero(as_tuple=False).squeeze(-1)
    env.object_dof_vel[:] = 0
    env.object_linvel[:] = 0
    env.object_angvel[:] = 0
    time.sleep(0.01)
    if done.any():
        env.reset_idx(env_ids, goal_env_ids)

In [29]:
env.shadow_hand_dof_pos, env.shadow_hand_dof_default_pos

(tensor([[-0.0703, -0.0729, -0.0570,  3.1286,  3.1424,  3.1330,  0.5274,  1.3452,
           1.2076,  1.7161,  0.5287,  1.4745,  1.1108,  1.6878,  0.5330,  1.4732,
           1.1114,  1.6875,  1.3294,  1.1439,  0.9266,  1.7187]]),
 tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]))

In [24]:
transform = gym.get_viewer_camera_transform(env.viewer, env.envs[0])

camera_pos = gymapi.Vec3(0.2, -0.35, 0.42)
camera_look_at = gym.get_env_origin(env.envs[0])
gym.viewer_camera_look_at(env.viewer, env.envs[0], camera_pos, camera_look_at)

In [ ]:
n_eps = 0
total_eps = 10
obs, _ = agent.env.reset_done()
batch_size = agent.get_batch_size(obs['obs'], 1)
obses = []
rewards = []
infos = []

while n_eps < total_eps:
    actions = agent.get_action(agent.obs_to_torch(obs))
    obs, r, done, info = agent.env_step(agent.env, actions.to(agent.env.device))
    obses.append(obs)
    rewards.append(r)
    infos.append(info)
    n_eps += done.sum()
    if done.any():
        obs, _ = agent.env.reset_done()

In [ ]:
max([info['task_dist'] for info in infos])

In [ ]:
pcts = []
for i, info in enumerate(infos):
    pcts.append(
        info['hand_dist'].cpu() * agent.env.dist_reward_scale / rewards[i].cpu())

In [ ]:
torch.mean(torch.cat(pcts))

## Test run one step
```python
obs = agent.env_reset(agent.env)
batch_size = agent.get_batch_size(obs['obs'], 1)
actions = agent.get_action(agent.obs_to_torch(obs))
obses, r, done, info = agent.env_step(agent.env, actions)
```